
# Cancer Risk Classification – Enhanced ANN (TensorFlow/Keras)

Notebook para o trabalho prático de **Inteligência Artificial** usando o dataset
`classificacao_risco_cancer_varios_valores.csv`.

Pipeline implementado:

- Upload do CSV via widget
- Limpeza e normalização semântica das variáveis
- Engenharia de atributos (scores de sintomas, hábitos e risco)
- Balanceamento com **SMOTE**
- Padronização dos atributos numéricos
- Rede Neural Profunda em **TensorFlow/Keras** (BatchNorm + Dropout)
- Validação cruzada estratificada **k-fold** com `k = 50`
- Cálculo das acurácias por fold, média e desvio padrão


## 1. Imports e configuração

In [ ]:
import io
import numpy as np
import pandas as pd

import ipywidgets as widgets
from IPython.display import display

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE

import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


## 2. Upload do arquivo CSV

In [ ]:
uploader = widgets.FileUpload(accept='.csv', multiple=False)
display(uploader)


In [ ]:
# Ler o CSV enviado via widget
if len(uploader.value) == 0:
    raise ValueError("Nenhum arquivo enviado. Faça o upload do CSV no widget acima.")

file_info = list(uploader.value.values())[0]
content = file_info['content']

df = pd.read_csv(io.BytesIO(content))
print("Formato do dataset:", df.shape)
df.head()


## 3. Limpeza básica e normalização semântica

In [ ]:
df_clean = df.copy()

# Mapas semânticos para categorias comuns
maps_yesno = {
    'Yes': 2,
    'No': 0,
    'Sometimes': 1,
    'Occasionally': 1,
    'None': 0,
    'Present': 1,
    'Absent': 0,
    'Always': 2,
    'never': 0
}

maps_level = {
    'Low': 0,
    'low': 0,
    'Medium': 1,
    'medium': 1,
    'Moderate': 1,
    'moderate': 1,
    'High': 2,
    'high': 2
}

# Substituir strings de nulo por NaN
null_str = ["NaN", "nan", "NONE", "None", "?", "null", "NULL", " "]
df_clean = df_clean.replace(null_str, np.nan)

# Aplicar mapas de categorias
for col in df_clean.columns:
    df_clean[col] = df_clean[col].replace(maps_yesno)
    df_clean[col] = df_clean[col].replace(maps_level)

# Remover ID se existir
if "Patient Id" in df_clean.columns:
    df_clean = df_clean.drop(columns=["Patient Id"])

df_clean.head()


## 4. Definição da variável alvo e tratamento de faltantes

In [ ]:
if "Level" not in df_clean.columns:
    raise ValueError("A coluna 'Level' não foi encontrada no dataset.")

# Remover linhas sem alvo
df_clean = df_clean.dropna(subset=["Level"])

# Converter remanescentes para numérico quando possível
for col in df_clean.columns:
    if df_clean[col].dtype == "object":
        try:
            df_clean[col] = df_clean[col].astype(float)
        except:
            # Codificar categorias restantes como códigos inteiros
            df_clean[col] = df_clean[col].astype("category").cat.codes

# Imputar faltantes com mediana nas colunas numéricas
df_clean = df_clean.fillna(df_clean.median(numeric_only=True))

X = df_clean.drop(columns=["Level"])
y = df_clean["Level"].astype(int)

print("Formato de X:", X.shape)
print("Formato de y:", y.shape)
y.value_counts(normalize=True)


## 5. Engenharia de atributos (scores agregados)

In [ ]:
# Heurísticas simples para agrupar colunas
symptom_cols = [c for c in X.columns if 'Cough' in c or 'Fatigue' in c or 'Pain' in c or 'breath' in c.lower()]
habits_cols = [c for c in X.columns if 'smok' in c.lower() or 'alcohol' in c.lower() or 'obesity' in c.lower()]
risk_cols = [c for c in X.columns if 'Pollution' in c or 'Hazards' in c or 'Risk' in c]

print("Colunas de sintomas:", symptom_cols)
print("Colunas de hábitos:", habits_cols)
print("Colunas de risco:", risk_cols)

X = X.copy()
X["symptom_score"] = X[symptom_cols].sum(axis=1) if symptom_cols else 0
X["habits_score"] = X[habits_cols].sum(axis=1) if habits_cols else 0
X["risk_score"] = X[risk_cols].sum(axis=1) if risk_cols else 0

X.head()


## 6. Balanceamento das classes com SMOTE

In [ ]:
sm = SMOTE(random_state=SEED)
X_sm, y_sm = sm.fit_resample(X, y)

print("Formato após SMOTE:", X_sm.shape)
y_sm.value_counts()


## 7. Padronização dos atributos (StandardScaler)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sm)

X_scaled.shape


## 8. Definição da Rede Neural Profunda (Keras)

In [ ]:
def build_model(input_dim, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Modelo definido.")


## 9. Validação cruzada estratificada k-fold (k = 50)

In [ ]:
k = 50
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=SEED)

scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y_sm), start=1):
    print(f"===== Fold {fold}/{k} =====")
    Xtr, Xval = X_scaled[train_idx], X_scaled[val_idx]
    ytr, yval = y_sm.iloc[train_idx], y_sm.iloc[val_idx]

    model = build_model(Xtr.shape[1], len(np.unique(y_sm)))

    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True
    )

    model.fit(
        Xtr, ytr,
        validation_data=(Xval, yval),
        epochs=80,
        batch_size=32,
        verbose=0,
        callbacks=[es]
    )

    loss, acc = model.evaluate(Xval, yval, verbose=0)
    print("Acurácia:", acc)
    scores.append(acc)

scores = np.array(scores)
print("\nAcurácias:", list(scores))
print("Média:", scores.mean())
print("Desvio:", scores.std())


## 10. Observações

- O bloco acima imprime:
  - Acurácia em cada um dos 50 folds
  - A acurácia média na validação cruzada
  - O desvio padrão entre os folds

- Estes valores podem ser mencionados no relatório da disciplina, bem como
  as escolhas de pré-processamento, engenharia de atributos e arquitetura da rede.
